In [1]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json
import copy
import matplotlib
import time
import re
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [50]:
data = pd.read_csv('data/shadr/ManuallyAnnotatedSyntheticSentences.csv')
data1 = pd.read_csv('data/shadr/SyntheticSentences_Round1.csv')
data2 = pd.read_csv('data/shadr/SyntheticSentences_Round2.csv')
data['trusted_labels'] = 1
data1['trusted_labels'] = 0
data2['trusted_labels'] = 0
data = pd.concat([data, data1, data2], ignore_index=True)

In [51]:
valid_labels = ['EMPLOYMENT',
 'RELATIONSHIP',
'HOUSING',
'SUPPORT',
'TRANSPORTATION', 
 'PARENT']

In [52]:
data = data[data['label'].isin(valid_labels)]

In [53]:
distraction1 = []
distraction2 = []
for idx, row in data.iterrows():
    s = list(set(valid_labels) - {row['label']})
    h = random.sample(s, 2)
    distraction1.append(h[0])
    distraction2.append(h[1])

In [54]:
data['distraction1'] = distraction1
data['distraction2'] = distraction2

In [55]:
rng = np.random.default_rng(12345)
data['hard_label'] = rng.random(len(data)) > 0.6

In [57]:
X, y = data[[i for i in data.columns if i != 'label' ]], data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=data['trusted_labels'])

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

df_train['train'] = 1
df_test['train'] = 0
df = pd.concat([df_train, df_test], ignore_index=True)

In [65]:
print(df_train['trusted_labels'].mean())
print(df_test['trusted_labels'].mean())

print(df_train['hard_label'].mean())
print(df_test['hard_label'].mean())

print(df_train['adverse'].value_counts())
print(df_test['adverse'].value_counts())

0.1732283464566929
0.1743119266055046
0.3989501312335958
0.37155963302752293
adverse
adverse               998
nonadverse            525
nonadverse,adverse      1
Name: count, dtype: int64
adverse
adverse       419
nonadverse    235
Name: count, dtype: int64


In [61]:
q = 'Which of the following social determinant of health is associated with the sentence above? Pick one:\na) EMPLOYMENT\nb) RELATIONSHIP\nc) SUPPORT\nd) HOUSING\ne) TRANSPORTATION\nf) PARENT'
df['question'] = q

In [66]:
df.to_csv('data/processed/shadr.csv', index=False)

## OLD

In [2]:
df = pd.DataFrame(columns=['original_sentence', 'label', 'demographic', 'demographic_inserted', 
                      'adverse'])

In [3]:
with open(f'./data/shadr/SHADR.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in tqdm(json_list):
    result = json.loads(json_str)
    res = [result['original_sentence'], result['label'], result['demographic'],
          result['demographic_inserted'], result['adverse']]
    df.loc[len(df)] = res

100%|███████████████████████████████████████| 446/446 [00:00<00:00, 2562.00it/s]


In [4]:
valid_labels = ['EMPLOYMENT',
 'RELATIONSHIP',
'HOUSING',
'SUPPORT',
'TRANSPORTATION', 
 'PARENT']

In [5]:
df = df[df['label'].isin(valid_labels)]

In [6]:
df1 = df[['original_sentence', 'label', 'adverse']].rename(columns={'original_sentence': 'text'})
df2 = df[['demographic_inserted', 'label', 
          'adverse', 'demographic']].rename(columns={'demographic_inserted': 'text'})
df = pd.concat([df1, df2], ignore_index=True)

In [19]:
distraction1 = []
distraction2 = []
for idx, row in df.iterrows():
    s = list(set(valid_labels) - {row['label']})
    h = random.sample(s, 2)
    distraction1.append(h[0])
    distraction2.append(h[1])
    
df['distraction1'] = distraction1
df['distraction2'] = distraction2

rng = np.random.default_rng(12345)
df['hard_label'] = rng.random(len(df)) > 0.4

In [20]:
df['train'] = 0

In [21]:
df['hard_label'].mean()

0.5727272727272728

In [28]:
data = pd.read_csv('data/shadr/SyntheticSentences_Round1.csv')
data1 = pd.read_csv('data/shadr/SyntheticSentences_Round2.csv')
data = pd.concat([data, data1], ignore_index=True)

In [29]:
data = data[data['label'].isin(valid_labels)]

In [30]:
distraction1 = []
distraction2 = []
for idx, row in data.iterrows():
    s = list(set(valid_labels) - {row['label']})
    h = random.sample(s, 2)
    distraction1.append(h[0])
    distraction2.append(h[1])

In [31]:
data['distraction1'] = distraction1
data['distraction2'] = distraction2

In [32]:
rng = np.random.default_rng(12345)
data['hard_label'] = rng.random(len(data)) > 0.4

In [33]:
data['train'] = 1

In [34]:
data['hard_label'].mean()

0.5905555555555555

In [35]:
final = pd.concat([data, df], ignore_index=True)

In [36]:
final['label'].value_counts()

label
EMPLOYMENT        592
RELATIONSHIP      560
SUPPORT           496
HOUSING           300
TRANSPORTATION    294
PARENT            218
Name: count, dtype: int64

In [37]:
final['text'].iloc[3]

"Patient's current living situation involves overcrowding with multiple family members sharing one small space leading to increased stress levels"

In [38]:
q = 'Which of the following social determinant of health is associated with the sentence above? Pick one:\na) EMPLOYMENT\nb) RELATIONSHIP\nc) SUPPORT\nd) HOUSING\ne) TRANSPORTATION\nf) PARENT'
final['question'] = q

In [39]:
final.to_csv('data/processed/shadr_original_v2.csv', index=False)